In [1]:
import polars as pd

APPLE = '/kaggle/input/apple-quality/apple_quality.csv'

# we have some weird junk at the end of this file so we need to load an explicit number of rows
df = pd.read_csv(source=APPLE, n_rows=3941)

df.head()

A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
i64,f64,f64,f64,f64,f64,f64,f64,str
0,-3.970049,-2.512336,5.34633,-1.012009,1.8449,0.32984,-0.49159,"""good"""
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.86753,-0.722809,"""good"""
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,"""bad"""
3,-0.657196,-2.271627,1.324874,-0.097875,3.63797,-3.413761,0.790723,"""good"""
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,"""good"""


In [2]:
df['Quality'].value_counts()

Quality,count
str,u32
"""bad""",1968
"""good""",1973


In [3]:
from umap import UMAP
from plotly.express import scatter

reducer = UMAP(n_components=2, random_state=2024, transform_seed=2024, verbose=True, n_jobs=1, n_epochs=200)

reducer_df = pd.DataFrame(data=reducer.fit_transform(X=df.drop(columns=['Quality'])), schema={'ux': pd.Float32, 'uy': pd.Float32})
# reducer_df.with_columns(target = pd.Series(df['Quality']))
# reducer_df['target'] = df['Quality'].copy() # .map({'good': True, 'bad': False})
scatter(data_frame=reducer_df, x='ux', y='uy', color=df['Quality'], height=900)


UMAP(n_epochs=200, n_jobs=1, random_state=2024, transform_seed=2024, verbose=True)
Mon Jan 29 17:08:22 2024 Construct fuzzy simplicial set
Mon Jan 29 17:08:30 2024 Finding Nearest Neighbors
Mon Jan 29 17:08:35 2024 Finished Nearest Neighbor Search
Mon Jan 29 17:08:39 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Mon Jan 29 17:08:44 2024 Finished embedding
